# Deepcamp: Codelab 4

**In this tutorial we will cover**:

- Large Language Models (ever heard of ChatGPT 👀 ?)
- LangChain: a tool to allow interaction between LLMs


**Author**:
- Alessio Devoto (alessio.devoto@uniroma1.it)


**Duration**: 50 mins 


**Warning**: Make sure you have an OpenAI token.

# Large language models with LangChain

1. Set env and try some API requests
2. Add some element to the chain and play around a bit
3. Question answering given prompt: the problem of context
3. Question answering over docs